In [1]:
# Notebook: Generate Notable State Correlation Trends
import pandas as pd

# --- Step 1: Load Cleaned Dataset ---
print("Loading cleaned dataset...")
data_path = "data/cleaned_multiple_cause_of_death.csv"
df = pd.read_csv(data_path)

# Convert necessary columns to numeric
numeric_columns = ["Deaths", "Population", "Crude Rate", 
                   "Prescriptions Dispensed by US Retailers in that year (millions)"]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Sort by state and year
df_sorted = df.sort_values(by=["State", "Year"]).copy()

# Compute Year-over-Year percentage changes for each metric
for col in numeric_columns:
    df_sorted[f"{col} Change (%)"] = df_sorted.groupby("State")[col].pct_change() * 100

# Drop first year for each state
df_change = df_sorted.dropna()

# --- Step 2: Compute Correlation Trends ---
print("Computing correlation trends for each state...")
correlation_results = []
states = df_change["State"].unique()
metrics = [col for col in df_change.columns if "Change (%)" in col]

# Define the valid metric pairs to keep (both orientations allowed)
valid_metric_pairs = {
    ("Population Change (%)", "Deaths Change (%)"),
    ("Prescriptions Dispensed by US Retailers in that year (millions) Change (%)", "Deaths Change (%)"),
    ("Population Change (%)", "Prescriptions Dispensed by US Retailers in that year (millions) Change (%)"),
    ("Population Change (%)", "Crude Rate Change (%)"),
    ("Prescriptions Dispensed by US Retailers in that year (millions) Change (%)", "Crude Rate Change (%)")
}

for state in states:
    state_df = df_change[df_change["State"] == state].drop(columns=["State", "Year"])
    state_corr = state_df.corr().stack().reset_index()
    state_corr.columns = ["Metric 1", "Metric 2", "Correlation"]
    state_corr["State"] = state

    # Ensure only valid metric pairs are included, accounting for both orientations
    state_corr = state_corr[state_corr.apply(lambda row: 
        (row["Metric 1"], row["Metric 2"]) in valid_metric_pairs or 
        (row["Metric 2"], row["Metric 1"]) in valid_metric_pairs, axis=1)]
    
    correlation_results.append(state_corr)

# Combine all state correlation results
df_notable_corr = pd.concat(correlation_results, ignore_index=True)

# Save filtered correlation trends dataset
df_notable_corr.to_csv("data/notable_state_correlation_trends_filtered.csv", index=False)

print("Filtered notable state correlation trends dataset generated successfully!")




Loading cleaned dataset...
Computing correlation trends for each state...


C:\Users\nicar\AppData\Local\Temp\ipykernel_3356\3589182503.py:20: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_sorted[f"{col} Change (%)"] = df_sorted.groupby("State")[col].pct_change() * 100
C:\Users\nicar\AppData\Local\Temp\ipykernel_3356\3589182503.py:20: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_sorted[f"{col} Change (%)"] = df_sorted.groupby("State")[col].pct_change() * 100


Filtered notable state correlation trends dataset generated successfully!


In [2]:
# --- Step 3: Identify Missing States at Each Stage ---
print("\nChecking for missing states during processing...")

# Load original cleaned dataset
df_cleaned = pd.read_csv("data/cleaned_multiple_cause_of_death.csv")
original_states = set(df_cleaned["State"].unique())

# Extract states from dataset after computing percentage changes
df_change_states = set(df_change["State"].unique())

# Extract states from final correlation results
df_notable_corr_states = set(df_notable_corr["State"].unique())

# Identify missing states at each stage
missing_after_change = original_states - df_change_states
missing_after_correlation = df_change_states - df_notable_corr_states

# Print results
if missing_after_change:
    print("States missing after computing percentage changes:")
    print(", ".join(sorted(missing_after_change)))
else:
    print("No states were lost after computing percentage changes.")

if missing_after_correlation:
    print("States missing after correlation filtering:")
    print(", ".join(sorted(missing_after_correlation)))
else:
    print("No states were lost after correlation filtering.")

print("\nProcessing completed successfully!")



Checking for missing states during processing...
No states were lost after computing percentage changes.
States missing after correlation filtering:
North Dakota

Processing completed successfully!
